### 1. Про корпус

корпус и описание трудных моментов -- в файле 

корпус с разметкой, который я использовала для золотого стандарта -- в файле 'copied'

In [670]:
!pip install pymorphy
!pip install spacy
!python -m spacy download ru_core_news_md
!pip install stanza
import stanza
import spacy
from pymorphy2 import MorphAnalyzer
import nltk

[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [671]:
with open('text', 'r', encoding = 'utf-8') as f:
    text = f.read()

### 2. Три теггера

In [672]:
# списки для тегов
pos_pymorphy = []
pos_spacy = []
pos_stanza = []

In [673]:
# pymorphy
tokenized = nltk.word_tokenize(text)
morph = MorphAnalyzer()
for token in tokenized:
    pos = morph.parse(token)[0].tag.POS
    if pos != None:
        # pymorphy выделяет отдельны1 тег для сравнительной степени
        if pos == "COMP":     # перевожу теги COMP в наречие/прилагательное
            g = morph.parse(token)[0].tag.gender
            if g == 'masc' or g == 'femn' or g == 'neut':  
                pos = 'ADJF'
            else:
                pos = 'ADVB'
        pos_pymorphy.append(pos)

In [674]:
# spacy
m_sp = spacy.load('ru_core_news_md')
text_sp = m_sp(text)
for token in text_sp:
    if token.pos_ != 'PUNCT':            
        pos_spacy.append(token.pos_) 

In [675]:
# stanza
m_st = stanza.Pipeline('ru', processors='tokenize,pos')
text_st = m_st(text)
for sentence in text_st.sentences:
    for token in sentence.words:
        if token.pos != 'PUNCT':
            pos_stanza.append(token.pos)

2022-10-11 22:49:55 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-11 22:49:57 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2022-10-11 22:49:57 INFO: Use device: cpu
2022-10-11 22:49:57 INFO: Loading: tokenize
2022-10-11 22:49:57 INFO: Loading: pos
2022-10-11 22:49:57 INFO: Done loading processors!


In [676]:
# золотой стандарт (список тегов)
with open('copied', 'r', encoding='utf-8') as f2:
    allt = f2.read()
pos_gold = list(allt.split('\n'))[1::2][:-1]

### 3. Сведение к одному стандарту, оценка accuracy

In [677]:
for i, p in enumerate(pos_pymorphy):
    if p == 'ADJF':
        pos_pymorphy[i] = 'ADJ'
    if p == 'ADVB':
        pos_pymorphy[i] = 'ADV'
    if p == 'GRND':
        pos_pymorphy[i] = 'VERB'
    if p == 'INFN':
        pos_pymorphy[i] = 'VERB'

In [678]:
for i, p in enumerate(pos_spacy):
    if p == 'DET':
        pos_spacy[i] = 'ADJ'
    if p == 'CCONJ':
        pos_spacy[i] = 'CONJ'
    if p == 'SCONJ':
        pos_spacy[i] = 'CONJ'
    if p == 'PRON':
        pos_spacy[i] = 'NPRO'
    if p == 'PART':
        pos_spacy[i] = 'PRCL'
    if p == 'ADP':
        pos_spacy[i] = 'PREP'
    if p == 'AUX':
        pos_spacy[i] = 'VERB'
    if p == 'PROPN':
        pos_spacy[i] = 'NOUN'

In [679]:
for i, p in enumerate(pos_stanza):
    if p == 'DET':
        pos_stanza[i] = 'ADJ'
    if p == 'CCONJ':
        pos_stanza[i] = 'CONJ'
    if p == 'SCONJ':
        pos_stanza[i] = 'CONJ'
    if p == 'PRON':
        pos_stanza[i] = 'NPRO'
    if p == 'PART':
        pos_stanza[i] = 'PRCL'
    if p == 'ADP':
        pos_stanza[i] = 'PREP'
    if p == 'PROPN':
        pos_stanza[i] = 'NOUN'

In [680]:
from sklearn.metrics import accuracy_score

In [681]:
accuracy_score(pos_gold, pos_pymorphy)

0.88

In [682]:
accuracy_score(pos_gold, pos_spacy)

0.9155555555555556

In [683]:
accuracy_score(pos_gold, pos_stanza)

0.9111111111111111

самая высокая accuracy -- у spacy, он используется в следующем пункте

### 4. Создание чанкера
Используются шаблоны:
1) не + ADJ; не + VERB -- качество должно улучшиться, потому что учитывается отрицание

2) нет + NOUN -- качество должно улучшиться, потому что учитывается отрицание

3) VERB + но; ADJ + но -- качество должно улучшиться, потому что после но обычно идет противоположная информация (тоже вроде учета отрицания)

In [684]:
def my_chunker(text):
    tokens = {}
    poses = {}
    text_sp = m_sp(text)
    c = 0
    chunks_adjs = []
    chunks_verbs = []
    chunks_nouns = []
    but_verbs = []
    but_adjs = []
    for token in text_sp:
        if token.pos_ != 'PUNCT':            
            tokens[c] = token
            poses[c] = [token.pos_]
            c += 1
    for k in range(len(tokens.keys()) - 1):
        if (str(tokens[k]) == 'не') and (str(poses[k+1][0]) == 'ADJ'):
            chunk = str(tokens[k]) + ' ' + str(tokens[k+1])
            chunks_adjs.append(chunk)
        if (str(tokens[k]) == 'не') and (str(poses[k+1][0]) == 'VERB'):
            chunk = str(tokens[k]) + ' ' + str(tokens[k+1])
            chunks_verbs.append(chunk)
        if (str(tokens[k]) == 'нет') and (str(poses[k+1][0]) == 'NOUN'):
            chunk = str(tokens[k]) + ' ' + str(tokens[k+1])
            chunks_nouns.append(chunk)
        if (str(poses[k][0]) == 'VERB') and (str(tokens[k+1]) == 'но'):
            chunk = str(tokens[k]) + ' ' + str(tokens[k+1])
            but_verbs.append(chunk)
        if (str(poses[k][0]) == 'ADJ') and (str(tokens[k+1]) == 'но'):
            chunk = str(tokens[k]) + ' ' + str(tokens[k+1])
            but_adjs.append(chunk)
    chunks = chunks_adjs + chunks_verbs + chunks_nouns + but_verbs + but_adjs
    return chunks

### 4 Изменение первой домашки

как было 

In [685]:
import requests
from bs4 import BeautifulSoup
from string import punctuation
import re
import nltk
from pymorphy2 import MorphAnalyzer

In [686]:
links_neg = ['https://otzov-mf.ru/napitok-energeticheskij-red-bull-otzyvy/', 
             'https://otzov-mf.ru/energeticheskij-napitok-flash-energy-otzyvy/', 
             'https://otzov-mf.ru/napitok-energeticheskij-strike-otzyvy/',
             'https://otzov-mf.ru/energeticheskij-napitok-ten-strike-sky-otzyvy/']


links_pos = ['https://otzov-mf.ru/napitok-energeticheskij-red-bull-otzyvy/', 
             'https://otzov-mf.ru/energeticheskij-napitok-flash-energy-otzyvy/', 
             'https://otzov-mf.ru/napitok-energeticheskij-strike-otzyvy/']
# 33 отрицательных, 32 положительных

text_neg = ''
text_pos = ''

for ln in links_neg:
    page = requests.get(ln)
    text = page.text
    soup = BeautifulSoup(text, 'html.parser')
    soup_neg = soup.find('div',{'class':'itric'})
    texts_neg = soup_neg.find_all('p')
    for t in texts_neg:
        text_neg = text_neg + ' ' + t.text.strip().strip(punctuation) + " "


link_patt = re.compile('ht[tps]+://[^А-я]+')
links = re.findall(link_patt, text_neg)
for l in links:
    text_neg = text_neg.replace(l, '')

In [687]:
l_neg = nltk.word_tokenize(text_neg)
words_neg = [word.lower() for word in l_neg if word.isalpha()]
lemmas_neg = []
morph = MorphAnalyzer()
for form in words_neg:
    first = morph.parse(form)[0]
    lemmas_neg.append(first.normal_form)

In [688]:
for lp in links_pos:
    page = requests.get(lp)
    text = page.text
    soup = BeautifulSoup(text, 'html.parser')
    soup_pos = soup.find('div',{'class':'pozitive'})
    texts_pos = soup_pos.find_all('p')
    for t in texts_pos:
        text_pos = text_pos + ' ' + t.text.strip().strip(punctuation) + " "

link_patt = re.compile('ht[tps]+://[^А-я]+')
links = re.findall(link_patt, text_pos)
for l in links:
    text_pos = text_pos.replace(l, '')
    text_pos = text_pos.replace('http://відгук.укр/ru/напиток_энергетический_red_bull-r129764.html', '')   

In [689]:
l_pos = nltk.word_tokenize(text_pos)
words_pos = [word.lower() for word in l_pos if word.isalpha()]
lemmas_pos = []
morph = MorphAnalyzer()
for form in words_pos:
    first = morph.parse(form)[0]
    lemmas_pos.append(first.normal_form)

In [690]:
from collections import Counter
counter_pos = Counter(lemmas_pos)
counter_pos2 = {k:counter_pos[k] for k in counter_pos if counter_pos[k] > 4}

In [691]:
counter_neg = Counter(lemmas_neg)
counter_neg2 = {k:counter_neg[k] for k in counter_neg if counter_neg[k] > 4}

In [692]:
set_pos = set(counter_pos2.keys())
set_neg = set(counter_neg2.keys())
only_pos = set_pos - set_neg
only_neg = set_neg - set_pos

In [693]:
def ton_de(text):
    cp = 0
    cn = 0
    words = nltk.word_tokenize(text)
    for w in words:  # для слов 
        if w in only_pos:
            cp += 1
        if w in only_neg:
            cn += 1
    
    
    
    t_for_b = [word.lower().strip(punctuation) for word in text.split(' ') if word.isalpha()] # для биграм
    bigrams = nltk.bigrams(t_for_b)
    my_bigrams = []
    for b in list(bigrams):
        list_b = []
        for i in range(len(list(b))):
            first = morph.parse(list(b)[i])[0]
            lemma = first.normal_form
            list_b.append(lemma)
            lemmas = ' '.join(list_b)
        my_bigrams.append(lemmas)
    for bigram in my_bigrams:
        if bigram in only_pos:
            cp += 1
        if bigram in only_neg:
            cn += 1
            
    if cn > cp:
        return 0
    else:
        return 1

In [694]:
reviews_test = ['''Доброго времени суток обитателям Айрекоменда
Я все еще нахожусь в поисках идеального энергетика для себя, хоть и понимаю, что пользы от таких напитков никакой нет, но все знают ситуации, когда нужно успеть сделать миллион дел, но энергии совсем нет. Сегодня речь пойдет об энергетике Торнадо Актив.
Место покупки - супермаркет "Ярче!"
Цена - 30 рублей
Приехав с учебы я осознала, что энергии после 5 пар у меня не осталось, а впереди предстояла бессонная ночь в попытках написать хотя бы малую часть курсовой работы, думаю такая или подобная ситуация знакома многим. Зашла по пути домой в супермаркет и направилась к полке с энергетиками, и взяв пару разных направилась на кассу.
Придя домой я глазами пробежалась по упаковке
Состав не самый плохой.
Я открыла банку и уловила уже до боли знакомый аромат. Так пахнут большинство энергетиков.
Цвет - как у фанты
Вкус - ну как сказать... типичный для энергетика, ничего нового и необычного я не почувствовала. Газирован не сильно, слизистые не горят. Однако энергетического эффекта я тоже не заметила, опустошив банку я все так же хотела спать и была все такой же уставшей. Крайне не рекомендую этот энергетик, если уж травить организм, то тем энергетиком который работает, а не этой пустышкой
''', 
'''Достоинства:
Цена, и то по акции
Недостатки:
Вкус, состав, все одним словом
Энергетики до этого никогда не брала. Пару раз пробовала, не очень понравилось, сладко и бееее.
Но тут зашла в "Магазинчик", а там акция на него, ну и я такая: "Ну возьму, была ни была". Взяла, на кассе на меня ооочень косо смотрели но продали. По акции банка 0.45 л стоило, кажется, 28 или 29. Точно не помню)
Вышла я такая на каблуках, в платюшке 15 лет) и с банкой "Tornado" в руках)
О вкусе: слишком химически кисло, слишком апельсиново, слишком кисло-сладко
Мне не понравилось( выпила совсееем немного (просто пить хотелось очень
сильно) и выкинула гораздо больше половины.
Взяла и пожалела. Всем любителям "Tornado" -- советую, всем остальным -- не очень)
Извините, что так долго не появлялась( не было настроения, желания, бабушки болели, одна до сих пор в больнице лежит...
''',
'''Достоинства:
Их нет и быть не может
Недостатки:
Убивает организм эта дрянь
Подробности:
Я наблюдательный человек и очень критичный, я много времени гуляю с ребенком при хорошей погоде и замечаю, что каждый третий подросток сидит курит и пьет ягуар ну конечно делать им то нечего гроб ят свое здоровье и молодость, но у меня только один вопрос зачем?? ведь они так молоды, да я знаю многие читая это подумают да нашлась тоже тут советчица, но поймите придет время вы захотите семью, детей, но ваш ь организм уже не сможет справится со всем, не губ те себя этими ягуар ами, производителю все ровно вас ему интересны вышли деньги и его рейтенги и что бы ягуар об ходил с ему дешевле всякой дряни туда напихают, но сами они его не уплотребляют, задумайтесь прежде чем пить этот яд
''',
'''Мне до сих пор нравиться его вкус,такой резкий и немного кисловатый.Нравилось состояние после выпитой банки,-приподнятое и энергичное.Также привлекала и сама упаковка,немного шероховатая матовая.По доступной цене.Пока не начались проблемы с сердцем и многочисленные рассказы о побочных эффектах данного напитка.Самый запоминающийся был о том,что за ночь ЯГУАР разъедает марлю.Отказалась от него сразу,слава богу без серьезных проблем для здоровья.Знаю,что его давно запретили,но у нас он продается почти в каждом магазине(
''',
'''Ужасный напиток, который я не рокерскую употреблять никому, он очень сильно действует на организм человека, и от него можно отравится, я знаю много сотен людей которые отравились именно благодаря такому напитку, да стоит он не дорого, но какой вред он приносит организму вы сами подумайте. Что не стоит рисковать свое жизнью ради какого то напитка, который вам хочется выпить, не стоит над эти шутить, над своим здоровьем вообще не не стоит шутить, потому что это очень сильно. Я не понимаю тех людей которые вообще покупают этот напиток, в нем же нету ничего вкусного и полезного, он противный и не вкусный, я иногда просто не понимаю как его можно употреблять ,ведь он не полезен для организма, как его вообще пьют. Не понимаю,вот хоть сколько мне говори. А все равно никогда не пойму тех людей. Которые его употребляют, это не правильно, люди подумайте прежде чем брать данный напиток, подумайте прежде всего о своем здоровье, о том что вы все таки хотите жить а не умереть из за данного напитка, так что как напиток который может сломать жизнь я его не кому не рекомендую, потому что это действительно плохой напиток, который даже не стоит употреблять. И начинать его выпивать, люди подумайте прежде о своем здоровье, а потом уже употребляйте данный напиток.
''',
'''Достоинства:
Расслабляет, поднимает настроение
Недостатки:
Бывает изжога
Вкусный, с друзьями частенько его пьём и ничего, со здоровьем всё отлично. Очень расслабляет и поднимает настроение. Так что мне и знакомым нравится, не вижу ничего в нём плохого.
''',
'''Достоинства:
Вкус.Содержание витамина D.Запах.Приемлемая цена.
Недостатки:
Энергетики вредные
Всем читателям отзыва привет! Сегодня пишу отзыв на энергетический напиток Solar Power. Купил в Пятерочке. Цена за 0.5мл напитка составила 72 рубля -по акции. Цена за объем приемлемая.
Вкус грейпфрута, с небольшой кислинкой. В составе, по заверению производителя имеется витамин D, что не может не радовать. Думаю именно из-за содержания витамина D в составе, на энергетике присутствует изображение солнца. Запах цитрусовый, в нос не бьет.
Дизайн баночки приятный. Красивый шрифт, солнце с лучами, вообщем прикольно:D Такой энергетик, думаю, не будет сливаться на фоне других. Сама баночка на ощупь приятная, что тоже уходит в плюс.
Эффект бодрости по началу даже не заметил, но потом почувствовал. Он приходит со временем, как в энергетике- Жара, если это, конечно, не самовнушение.
Всем рекомендую к проде энергетический напиток Solar Power, так как он вкусный и бодрящий, а больше от энергетика и не нужно. Так же стоит помнить, что энергетические напитки вредят вашему здоровью, полезного в них мало. Не стоит с ними злоупотреблять!
''',
'''Достоинства:
Доступность Необычность
Недостатки:
Не обнаружила
Всем привет. Хочу поделиться своим отзывом на энергетический напиток соляр.
Как обычно начинаем с доступности. Есть во многих магазинах. Но не во всех. Оценка 7/10
Дизайн. Дизайн прикольный. Мне нравится. Но не самый лучший. Моя оценка 6/10
И так, цена. Цена на него везде какая то разная, но не больше сотки. В магните по скидону можно купить за 39 рублей. Ну 6/10 моя оценка.
И наконец самое интересное. Вкус. Вкус капец какой необычный, не похож ни на один энергетик. По вкусу это странно очень, зайдет далеко не всем, но мне понравилось. Послевкусие есть какого то пива что ли, хз. Блин, с кальмаром или сушёной рыбой это прям круто зайдет. Оценка 7/10
Ну общая оценка 6,5/10. Неплохо. По скидону попробуйте. Почему бы и нет.
''',
'''Достоинства:
бодрит сразу просыпаешься
Недостатки:
иногда и с него спать охота
Одно время я работал водителем такси и когда особенно ночью сильно устаешь или хочется сильно спать то ягуар очень выручал в этом вопросе. Мне кажется вкусный ягодный вкус совмещен с тонизирующими свойствами. Очень бодрит но перебарщивать с ним тоже не стоит пить как написано в инструкции.
''',
'''Люблю пить всякие разные энергетики, самый любимый это - Рэд Булл. Так что сравнение пойдет в параллель с ним. Как не странно вкус абсолютно идентичный, что конечно же делает этому напитку большой плюс. То есть он сладкий, и имеет приятную кислинку, никаких посторонних металлических привкусов не обнаружила.
Стоил он 29.90 за 0,5 - это очень отлично, по сравнению с другими ценами за такое количество, сам Рэд Булл стоит за 0,5 , если не ошибаюсь, рублей 80. Ничего особенного не ожидала от покупки, но была приятно удивлена и рада. Напиток очень вкусный, выбрала именно "Storm" потому что в его составе было написано меньше всего химической бяки, плюс на баночке заявлены витамины. А так, там было всего три вкуса - "storm"(красный), "ise"(голубой) "active"(оранжевый). Цвет напитка, честно, удивил, очень сильно напомнил пиво, но не тут-то было, кроме цвета, с пивом, этот напиток, не имеет, ровным счетом, ничего общего. По сравнению с прозрачно-желтым Рэдбулловским, этот же будет по-темнее, но ничего сверхъестественного в этом нет, цвет не противный и не специфичный. Похож даже на цвет газировки со вкусом "ирисок", если хотите. В меру газированный, пенка была, но не сильная.
В общем покупкой довольна, если увижу "Tornado energy", то обязательно куплю, попробую другие вкусы, потому что больше смысла переплачивать за Рэд Булл не вижу.
Всем рекомендую.
Что касается "прилива сил и энергии" не могу сказать, потому что энергетики на меня то ли не действуют, то ли я не замечаю, ну в общих чертах, после выпитой баночки носом не клевала до поздней ночи. Решающим фактором для меня при покупке энергетиков, является - вкус, а здесь он на высоте!
'''    
]

In [695]:
# для тестовых я просто накопипастила отзывы, сначала идут 5 отрицательных, потом 5 положительных
y_test = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [696]:
y_pred = []
for r in reviews_test:
    res = ton_de(r)
    y_pred.append(res)
y_pred

[0, 0, 0, 0, 0, 0, 1, 1, 0, 0]

In [697]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7

изменение 1 -- добавляем биграммы в множества положительных и отрицательных слов

In [698]:
chunks = my_chunker(' '.join(lemmas_neg) + ' '.join(lemmas_pos))

In [699]:
for c in chunks:
    spl = c.split(' ')
    if spl[1] in only_pos:
        only_neg.add(c)
    if spl[1] in only_neg:
        only_pos.add(c)      

изменение 2 -- изменяем функцию так, чтобы учитывались биграмы

In [700]:
def ton_de_new(text):
    cp = 0
    cn = 0
    words = nltk.word_tokenize(text)
    for w in words:  # для слов 
        if w in only_pos:
            cp += 1
        if w in only_neg:
            cn += 1
    
    
    
    t_for_b = [word.lower().strip(punctuation) for word in text.split(' ') if word.isalpha()] # для биграм
    bigrams = nltk.bigrams(t_for_b)
    my_bigrams = []
    for b in list(bigrams):
        list_b = []
        for i in range(len(list(b))):
            first = morph.parse(list(b)[i])[0]
            lemma = first.normal_form
            list_b.append(lemma)
            lemmas = ' '.join(list_b)
        my_bigrams.append(lemmas)
    for bigram in my_bigrams:
        if bigram in only_pos:
            cp += 1
        if bigram in only_neg:
            cn += 1
            
    if cn > cp:
        return 0
    else:
        return 1

In [701]:
y_pred2 = []
for r in reviews_test:
    res = ton_de_new(r)
    y_pred2.append(res)
y_pred
accuracy_score(y_pred2, y_test)

0.6

accuracy стала ниже..